# Import

In [567]:
import tensorflow_datasets as tfds
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout #type: ignore
from tensorflow.keras.optimizers import Adam #type: ignore
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical #type: ignore
import numpy as np
from tensorflow.keras.applications import ResNet50 #type: ignore
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions # type: ignore
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.applications.vgg16 import preprocess_input #type: ignore
from sklearn.metrics import confusion_matrix #type: ignore
import tensorflow.keras.backend as K #type: ignore

# Dataset

In [565]:
data, info = tfds.load("caltech101", with_info=True, as_supervised=True) #Carico il dataset da Tensorflow dataset

Splitto il dataset in train e test

In [566]:
train_data, test_data = data['train'], data['test']

Nomi delle classi

In [568]:
label_names = info.features['label'].names

Seleziono 5 classi

In [ ]:
selected_classes = [69, 96, 19, 16, 4]
class_map = {69: 0, 96: 1, 19: 2, 16: 3, 4: 4} #Le mappo per usarle nella rete neurale
print(selected_classes)

In [570]:
#Funzione per filtrare le immagini che appartengono alle classi selezionate
def filter_classes(image, label):
    return tf.reduce_any(tf.equal(label, selected_classes)) #Restituisce True se la classe è tra quelle selezionate

#Funzione per mappare le etichette originali in nuove etichette numeriche
def map_labels(image, label):
    label = tf.cast(label, tf.int64) #Ci assicutiamo che l'etichetta sia di tipo int64 altrimenti non e' compatibile

    #Rimpiazza l'etichetta originale con una nuova etichetta numerica
    label = tf.where(tf.equal(label, 69), tf.cast(0, tf.int64), label)
    label = tf.where(tf.equal(label, 96), tf.cast(1, tf.int64), label)
    label = tf.where(tf.equal(label, 19), tf.cast(2, tf.int64), label)
    label = tf.where(tf.equal(label, 16), tf.cast(3, tf.int64), label)
    label = tf.where(tf.equal(label, 4), tf.cast(4, tf.int64), label)

    return image, label 

#Filtra i dati di addestramento e test in modo che contengano solo le classi selezionate
train_data = train_data.filter(filter_classes)
test_data = test_data.filter(filter_classes)

#Applica la mappatura delle etichette ai dati di addestramento e test
train_data = train_data.map(map_labels)
test_data = test_data.map(map_labels)


Test se il filtro si è applicato

In [ ]:
for image, label in train_data.take(10):
    #Usiamo label_names per ottenere il nome della classe corrispondente
    print(f"Immagine con etichetta {label_names[label.numpy()]}")


Preprocessing delle immagini

In [572]:
#Funzione per preprocessare l'immagine e mantenerne l'etichetta
def preprocess_image(image, label):
    # Ridimensiona l'immagine che è la dimensione di input richiesta da ResNet50
    image = tf.image.resize(image, (224, 224))

    # Preprocessa l'immagine utilizzando la funzione di preprocessing specifica per il modello ResNet50 
    image = preprocess_input(image)

    return image, label

#Applica la funzione di preprocessamento a tutto il dataset di addestramento
train_data = train_data.map(preprocess_image)

#Applica la stessa funzione di preprocessamento a tutto il dataset di test
test_data = test_data.map(preprocess_image)


Batch e Prefatch

In [573]:
#Scegliamo la grandezza del batch size e la applichiamo al train e test
batch_size = 32
train_data = train_data.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Modello pre addestrato

In [574]:
# Funzione personalizzata per calcolare precision, recall e f1-score
def f1_metric(y_true, y_pred):
    #Convertiamo le predizioni in one-hot encoded
    y_pred_classes = K.argmax(y_pred, axis=-1)
    y_true_classes = K.cast(y_true, tf.int64)
    
    #Precision = TP / (TP + FP)
    true_positives = K.sum(K.cast(y_true_classes == y_pred_classes, tf.float32))
    predicted_positives = K.sum(K.cast(y_pred_classes, tf.float32))
    precision = true_positives / (predicted_positives + K.epsilon())

    #Recall = TP / (TP + FN)
    actual_positives = K.sum(K.cast(y_true_classes, tf.float32))
    recall = true_positives / (actual_positives + K.epsilon())

    #F1 Score = 2 * (Precision * Recall) / (Precision + Recall)
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1

Modello Pre-Addestrato

In [554]:
#Implementazione del modello resnet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

Congelo i pesi

Per il modello fine-tunato rimuovi i commenti del for

In [555]:
#Pesi congelati di tutto il modello
base_model.trainable = False

#Per sbloccare alcuni layer finali del modello pre-addestrato
#Questo è il modello fine-tunato
#for layer in base_model.layers[-5:]:
#    layer.trainable = True

Creiamo il nuovo modello

In [575]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(), #Riduce le feature map a un vettore 1D
    tf.keras.layers.Flatten(), #Appiattisce il vettore per i layer densi
    tf.keras.layers.Dropout(0.5), #Previne l'overfitting disattivando il 50% dei neuroni
    tf.keras.layers.Dense(128, activation='relu'), #Hidden layer con 128 neuroni
    tf.keras.layers.Dense(len(selected_classes), activation='softmax') #Previsione delle classi
])

Compilazione

In [576]:
model.compile(
    optimizer=Adam(learning_rate=0.001), #Modifico il learning rate di Adam
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy',f1_metric]
)

In [ ]:
model.summary()

Addestramento

In [ ]:
history = model.fit(train_data, validation_data=test_data, epochs=5)

# Plot risultati

Caricare il codice per printare i relativi grafici

In [ ]:
#Selezionare un interprete per eseguire il codice
#Grafico dell'andamento della Loss
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss durante il training')
plt.xlabel('Epoche')
plt.ylabel('Loss')
plt.legend()

#Grafico dell'andamento della val_accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy durante il training')
plt.xlabel('Epoche')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

#Grafico dell'andamento della Cross-Entropy Loss
plt.figure(figsize=(12, 6))

plt.plot(history.history['loss'], label='Train Cross-Entropy Loss')
plt.plot(history.history['val_loss'], label='Validation Cross-Entropy Loss')
plt.title('Cross-Entropy Loss durante il training')
plt.xlabel('Epoche')
plt.ylabel('Cross-Entropy Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Visualizzare le immagini con le rispettive predizioni

In [ ]:
#Funzione per visualizzare le immagini con le rispettive predizioni
def display_predictions(model, test_data, num_images=5):
    images, labels = next(iter(test_data))  #Prendi un batch
    predictions = model.predict(images)  #Predici le classi per il batch di immagini
    
    #Visualizza le prime 'num_images' immagini insieme alle loro etichette e predizioni
    plt.figure(figsize=(12, 12))
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.axis('off')
        
        #Trova la classe predetta
        predicted_class = np.argmax(predictions[i])
        
        #Mostra la classe reale e quella predetta
        plt.title(f"True: {label_names[labels[i].numpy()]}\nPred: {label_names[predicted_class]}")
    
    plt.tight_layout()
    plt.show()

#Chiamata della funzione per visualizzare le predizioni
display_predictions(model, test_data, num_images=5)

# Grafico per la visualizzazione della Confusion Matrix

In [561]:
#Funzione per calcolare e visualizzare la Confusion Matrix
def plot_confusion_matrix(model, test_data):
    true_labels = []
    pred_labels = []
    
    for images, labels in test_data:
        true_labels.extend(labels.numpy())
        predictions = model.predict(images)
        pred_labels.extend(np.argmax(predictions, axis=1))
    
    #Calcola la Confusion Matrix
    cm = confusion_matrix(true_labels, pred_labels)

    #Stampa la matrice di confusione nel terminale
    print("Confusion Matrix:")
    print(cm)

plot_confusion_matrix(model, test_data)

# Panoramica Generale

Ecco una panoramica delle principali fasi del processo e delle conclusioni che possiamo trarre:

1. Preparazione del Dataset:
- Abbiamo caricato il dataset Caltech-101 da TensorFlow Datasets e selezionato un sottoinsieme di 5 classi specifiche (classi 69, 96, 19, 16, 4).
- Le immagini sono state filtrate in base a queste classi selezionate e successivamente le etichette originali sono state mappate a etichette numeriche più compatibili con la rete neurale.
2. Preprocessing delle Immagini:
- Le immagini sono state ridimensionate a 224x224 pixel, che è la dimensione di input richiesta dal modello ResNet50.
- È stato applicato il preprocessing delle immagini utilizzando la funzione preprocess_input di ResNet50, che normalizza le immagini per adattarle al modello pre-addestrato.
3. Modello di Rete Neurale:
- Il modello utilizzato è basato su ResNet50, che è stato caricato senza la parte finale (inclusa la classificazione delle 1000 classi di ImageNet) grazie all'opzione include_top=False.
- Il modello è stato completato con un GlobalAveragePooling2D, un Flatten, un Dropout per prevenire l'overfitting, e due layer Dense finali per la classificazione.
- Il modello è stato compilato con l'ottimizzatore Adam e la sparse categorical cross-entropy come funzione di perdita, e come metrica l'accuratezza e il f1-score.
4. Addestramento del Modello:
- Il modello è stato addestrato per 5 epoche sui dati di addestramento e validato sui dati di test.
- Durante l'addestramento, sono stati tracciati i valori di loss e accuracy, che sono stati visualizzati tramite grafici per monitorare l'andamento del modello.
5. Valutazione del Modello:
- I risultati dell'addestramento sono stati mostrati in grafici che illustrano l'andamento della Loss e dell'Accuracy durante le epoche, permettendo di osservare il comportamento del modello su  entrambi i set di addestramento e validazione.
- Inoltre, il F1-score è stato calcolato per valutare il bilanciamento tra precisione e recall per ciascuna delle classi selezionate tuttavia presenta un valore insoddisfacente a causa dello sbilanciamento delle classi selezionate.
6. Visualizzazione delle Predizioni:
- Per meglio comprendere le prestazioni del modello, è stata creata una funzione per visualizzare alcune immagini di test con le relative etichette vere e predizioni. Questa fase permette di verificare visivamente se il modello è in grado di generalizzare correttamente su nuovi dati.
7. Matrice di Confusione:
- Una funzione è stata implementata per calcolare e visualizzare la matrice di confusione, che ci consente di analizzare gli errori di classificazione del modello. La matrice di confusione mostra, per ciascuna classe, quante volte il modello ha classificato correttamente o erroneamente le immagini.
8. Modello Finetunato:
- Infine abbiamo sbloccato i pesi degli ultimi 5 layer del modello pre-addestrato per permettere il cambio di pesi.
- Il modello aumenta drasticamente le performance raggiungendo il picco di metriche dopo solo 3 epoche.

### Conclusioni Finali

Il progetto ha avuto l'obiettivo di costruire, addestrare e confrontare un modello di classificazione delle immagini basato su **ResNet50** sui dati del dataset **Caltech-101**, selezionando specifiche classi e testando due approcci: uno con il modello pre-addestrato senza fine-tuning e l'altro con fine-tuning per adattare meglio il modello ai dati specifici.

#### 1. Prestazioni del Modello Pre-addestrato
Il modello ResNet50 pre-addestrato senza fine-tuning ha mostrato una buona capacità di generalizzazione, ma le performance non sono state ottimali. La **loss** e l'**accuracy** sui dati di test hanno indicato che il modello ha beneficiato del training iniziale su ImageNet, ma non è stato completamente adattato alle specifiche 5 classi selezionate del dataset Caltech-101. Nonostante ciò, il modello pre-addestrato ha comunque ottenuto risultati di classificazione ragionevoli.

#### 2. Fine-Tuning del Modello
Quando sono stati sbloccati gli ultimi layer del modello ResNet50 e il modello è stato ri-allenato sui dati di addestramento, i risultati sono migliorati drasticamente. Il **fine-tuning** ha permesso al modello di apprendere caratteristiche più specifiche per il nostro sottoinsieme di dati. Questo approccio ha portato a un aumento significativo dell'**accuracy** e a una riduzione della **loss** durante le epoche successive. In particolare, il modello ha raggiunto un picco nelle prestazioni dopo solo 3 epoche di addestramento, suggerendo che il fine-tuning ha avuto un impatto significativo nell'adattamento del modello.

#### 3. Analisi Visiva e Matrice di Confusione
La visualizzazione delle predizioni e la matrice di confusione hanno fornito un'ulteriore comprensione delle prestazioni del modello. Le immagini di test sono state correttamente classificate per la maggior parte delle volte, con alcuni errori evidenziati dalla matrice di confusione, in particolare per le classi meno rappresentate. Tuttavia, il fine-tuning ha portato a un miglioramento nella classificazione di alcune classi che inizialmente avevano risultati più scarsi.

#### 4. Confronto Pre-addestrato vs Fine-tuning
Il confronto tra il modello pre-addestrato e quello fine-tunato ha mostrato chiaramente che il fine-tuning consente al modello di adattarsi meglio al dataset specifico. La **loss** è stata significativamente inferiore e l'**accuracy** è aumentata, soprattutto durante le epoche successive. In generale, l'approccio di fine-tuning si è rivelato fondamentale per migliorare le performance, specialmente per un dataset che non è completamente simile al dataset su cui il modello è stato originariamente addestrato (ImageNet).

#### 5. Importanza del Preprocessing e delle Scelte del Modello
Il preprocessing delle immagini (ridimensionamento e normalizzazione) è stato un passo fondamentale per preparare i dati all'ingresso nel modello ResNet50, che ha richiesto un adattamento specifico per la dimensione dell'immagine e la normalizzazione dei pixel. Inoltre, la scelta di ResNet50 come modello di base si è rivelata ottimale per questa tipologia di task, grazie alla sua architettura profonda e pre-addestramento su un ampio dataset di immagini.

#### 6. Considerazioni Future
Nonostante i buoni risultati ottenuti, potrebbero essere esplorate ulteriori ottimizzazioni, come l'uso di tecniche avanzate di data augmentation, la sblocco di più strati per un fine-tuning più profondo, o l'uso di modelli più leggeri o specializzati per task di classificazione di immagini più mirati. Inoltre, l'analisi delle metriche come il **F1-score** e l'**accuracy** per ciascuna classe, potrebbe essere utile per capire meglio le aree in cui il modello ha bisogno di miglioramenti.

In conclusione, il progetto ha dimostrato che l'uso di modelli pre-addestrati come **ResNet50**, combinato con il fine-tuning, è un approccio potente ed efficace per la classificazione delle immagini, consentendo di ottenere ottime performance anche su un dataset specifico come Caltech-101. Il fine-tuning in particolare è essenziale per adattare modelli generali ai dati di dominio specifico, portando a miglioramenti significativi nelle prestazioni complessive.

# **Predizioni del modello Pre-addestrato**

# **Predizioni del modello Fine-tunato**